In [1]:
from copy import deepcopy
from functools import cache
from os import listdir
from genericpath import isfile
import json
import pickle
from datetime import datetime
import time
from functools import partial

from scipy.special import softmax
import numpy as np
import torch
import math
from itertools import combinations

import matplotlib.pyplot as plt
import matplotlib.animation as animation

from typing import Any, List, Tuple



# |Set root for GPUDrive import
import os
import sys
from pathlib import Path

from traitlets import default

# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive-CoDec':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)
sys.path.append(str(working_dir))


# |GPUDrive imports
from gpudrive.utils.config import load_config
from examples.CoDec_Research.code.gpuDrive_utils import get_gpuDrive_vars

# |Function to extract filename from path
env_path2name = lambda path: path.split("/")[-1].split(".")[0]



RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

2025-05-09 17:42:34.219716: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746826954.287346  423412 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746826954.307944  423412 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746826954.446766  423412 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746826954.446828  423412 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746826954.446829  423412 computation_placer.cc:177] computation placer alr

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

Load environments to get coordinate offsets

In [2]:
# |Set scenario path
dataset_path = 'data/processed/construal/Combined'

# |Model Config (on which model was trained)
training_config = load_config("examples/experimental/config/reliable_agents_params")

# |Set simulator config
max_agents = training_config.max_controlled_agents   # Get total vehicle count
training_config.max_controlled_agents = 1    # Control only the first vehicle in the environment
num_parallel_envs = 25
total_envs = 25
device = "cpu" # cpu just because we're in a notebook





### Instantiate Variables ###

env_config, train_loader, env, env_multi_agent, sim_agent = get_gpuDrive_vars(
                                                                                training_config = training_config,
                                                                                device = device,
                                                                                num_parallel_envs = num_parallel_envs,
                                                                                dataset_path = dataset_path,
                                                                                max_agents = max_agents,
                                                                                total_envs = total_envs,
                                                                                sim_agent_path= "daphne-cornelisse/policy_S10_000_02_27",
                                                                            )

sim_offsets_xyz = env.sim.world_means_tensor().to_torch().cpu().numpy()
env_databatch = [env_path2name(curr_scene_) for curr_scene_ in env.data_batch]

Get Server Construal Trajs

In [3]:
srFile = "examples/CoDec_Research/results/simulation_results/server/all_constr_obs_server.pickle"
with open(srFile, 'rb') as opn_file:
    traj_obs = pickle.load(opn_file)

# Visualization Logic (Dynamic)

In [4]:
def animate_scenario(curr_data: dict, filename: str):
    fig, ax = plt.subplots(1, 1, dpi=300)
    ax.set_xticks([])
    ax.set_yticks([])

    # |Plot roads
    min_x, min_y = (np.inf, np.inf)
    for road_obj in curr_data['roads']:
        road_xy = [(rdict['x'], rdict['y']) for rdict in road_obj['geometry']]
        color = {
            'crosswalk': 'grey',
            'driveway': 'grey',
            'lane': 'darkgrey',
            'road_edge': 'black',
            'road_line': 'yellow',
            'speed_bump': 'grey',
            'stop_sign': 'red'
        }[road_obj['type']]
        rx, ry = zip(*road_xy)
        if min(rx) < min_x: min_x = min(rx)
        if min(ry) < min_y: min_y = min(ry)
        ax.plot(rx, ry, color=color, lw=.2)
    ax.set_title(filename.split('/')[-1])

    # |Get vehicle info
    #2# |Loop through the rest
    for obj_num, veh_obj in enumerate(curr_data['objects']):
        veh_pos = [[posdict['x'],posdict['y']] if posdict['x']!=-10000 else [min_x,min_y] for posdict in veh_obj['position']]
        vx, vy = zip(*veh_pos)
        if obj_num == 0:
            all_vx = np.array([vx])
            all_vy = np.array([vy])
        else:
            all_vx = np.append(all_vx, [vx], axis=0)
            all_vy = np.append(all_vy, [vy], axis=0)

    # Set all vehicles except controlled vehicle to blue
    veh_colors = ['blue']*all_vx.shape[0]
    veh_colors[curr_data['metadata']['sdc_track_index']] = 'red'

    scat = ax.scatter(all_vx[:,0], all_vy[:,0], marker='o', s=veh_obj['width']*.1, lw=1, alpha=1, c=veh_colors)

    # |Plot veh traj
    def animate_func(timestep):        
        curr_offset = np.stack([all_vx[:,timestep],all_vy[:,timestep]]).transpose()
        scat.set_offsets(curr_offset)
    anim = animation.FuncAnimation(fig, animate_func, repeat=True, frames=all_vx.shape[1] - 1, interval=50)

    anim.save(filename=filename, fps=15)
    plt.close()


In [5]:
dirpath = "data/processed/construal/"
outputDir = "data/processed/Viz_construed_sim/"

for scene_num, (scene_name, scene_info) in enumerate(traj_obs.items()):
    if scene_name == 'dict_structure':
        continue
    # print(f"\rprocesseing file {scene_num}", end="", flush=True)
    with open(dirpath+scene_name+'.json', 'r') as opn_file:
        data = json.load(opn_file)
    for construal_indcs, construal_info in scene_info.items():
        for sample_num, sample_traj in construal_info.items():
            plot_title = scene_name+'_'+str(construal_indcs)+'_'+str(sample_num)
            sample_traj = sample_traj.cpu().numpy()
            # animate_scenario(data, outputDir+plot_title+'.gif', sample_traj, construal_indcs, (0), plot_title)
    

FileNotFoundError: [Errno 2] No such file or directory: 'data/processed/construal/tfrecord-00101-of-01000_71.json'

# Visualization Logic (Static)

In [7]:

def plot_scenario(curr_data: dict, 
                  filename: str,
                  sample_traj: np.array = None,
                  sim_offsets_xyz: np.array = None,
                  construal_indcs: tuple = None,
                  ego_indcs: tuple = None,
                  plot_title: str = None, 
                  save: bool = True,
                  fig: plt.figure = None,
                  ax: plt.axes = None,
                  ):
    """

    """
    if ax is None or fig is None:
        fig, ax = plt.subplots(1, 1, dpi=300)
        ax.set_xticks([])
        ax.set_yticks([])
        
    sim_offsetX, sim_offsetY, _ = sim_offsets_xyz

    # |Plot Roads
    lowerBound_x = float('inf')     # Later used as position filter
    upperBound_x = -float('inf')     # Later used as position filter
    for road_obj in curr_data['roads']:
        road_xy = [(rdict['x'], rdict['y']) for rdict in road_obj['geometry']]
        color = {
            'crosswalk': 'grey',
            'driveway': 'grey',
            'lane': 'darkgrey',
            'road_edge': 'black',
            'road_line': 'yellow',
            'speed_bump': 'grey',
            'stop_sign': 'red'
        }[road_obj['type']]
        rx, ry = zip(*road_xy)
        lowerBound_x = lowerBound_x if lowerBound_x < min(rx) else min(rx)
        upperBound_x = upperBound_x if upperBound_x > max(rx) else max(rx)
        ax.plot(rx, ry, color=color, lw=.2)
    ax.set_title(plot_title)

    # |Plot Vehicles
    if sample_traj is None:
        for veh_num, veh_obj in enumerate(curr_data['objects']):
            veh_traj = [[posdict['x'],posdict['y']] for posdict in veh_obj['position'] if posdict['x']!=-10000]
            vx, vy = zip(*veh_traj)
            if veh_num != curr_data['metadata']['sdc_track_index']:
                ax.plot(vx, vy, marker='.', ms=veh_obj['width']*.1, lw=1, alpha=0.3)
            else:
                ax.plot(vx, vy, marker='.', ms=veh_obj['width']*.1, lw=1, alpha=1, color='grey')
    else:
        construal_indcs = tuple(i for i in construal_indcs if i not in ego_indcs)    # Get rid of overlaps
        veh_width = curr_data['objects'][0]['width']*.1
        for veh_num, veh_traj in enumerate(sample_traj):
            veh_traj = veh_traj.tolist()
            veh_traj = np.array(veh_traj) + np.array([sim_offsetX, sim_offsetY])
            veh_traj = [curr_pos_ for curr_pos_ in veh_traj if lowerBound_x < curr_pos_[0] < upperBound_x]
            vx, vy = zip(*veh_traj)
            if veh_num in construal_indcs:
                ax.plot(vx, vy, marker='.', ms=veh_width, lw=1, alpha=0.5)
            elif veh_num in ego_indcs:
                ax.plot(vx, vy, marker='.', ms=veh_width, lw=1, alpha=0.2, color='red')
            
    # plt.show()
    if save:
        fig.savefig(filename)
        plt.close()
    else:
        return (fig, ax)


In [8]:
dirpath = "data/processed/construal/Combined/"
outputDir = "examples/CoDec_Research/results/simulation_results/Viz_construed_sim/"

for scene_num, scene_name in enumerate(env_databatch):
    scene_info = traj_obs.get(scene_name, None)
    if scene_info is None:
        continue
    print(f"\rprocesseing file {scene_num}", end="", flush=True)
    with open(dirpath+scene_name+'.json', 'r') as opn_file:
        data = json.load(opn_file)
    for construal_indcs, construal_info in scene_info.items():
        fig = None
        ax = None
        for sample_num, sample_traj in construal_info.items():
            plot_title = scene_name+'_'+str(construal_indcs)#+'_'+str(sample_num)
            sample_traj = sample_traj.cpu().numpy()
            fig, ax = plot_scenario(data, outputDir+plot_title+'.png', sample_traj, 
                                    sim_offsets_xyz[scene_num], construal_indcs, (0,), 
                                    plot_title, save=False, fig=fig, ax=ax)
            fig.savefig(outputDir+plot_title+'.png')
            plt.close()


processeing file 24

# Server Construal Values

In [ ]:
srFile = "examples/CoDec_Research/results/simulation_results/server/construal_vals_server.pickle"
with open(srFile, 'rb') as opn_file:
    default_values = pickle.load(opn_file)

In [4]:
default_values

{'dict_structure': '{scene_name: {construal_index: value}}',
 'tfrecord-00101-of-01000_71': {(0,): 1.0,
  (1,): 1.0,
  (5,): 1.0,
  (9,): 1.0,
  (10,): 1.0,
  (42,): 1.0,
  (43,): 1.0,
  (0, 1, 5, 9, 10, 42, 43): 1.0},
 'tfrecord-00117-of-01000_240': {(0,): 0.9555555555555556,
  (1,): 1.0,
  (2,): 1.0,
  (3,): 0.9,
  (4,): 0.9777777777777777,
  (5,): 0.9,
  (7,): 0.9111111111111111,
  (8,): 0.9444444444444444,
  (9,): 0.9555555555555556,
  (10,): 0.9777777777777777,
  (11,): 0.9222222222222223,
  (12,): 0.9777777777777777,
  (13,): 0.9333333333333333,
  (14,): 1.0,
  (15,): 1.0,
  (16,): 0.9777777777777777,
  (17,): 0.9333333333333333,
  (0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17): 1.0},
 'tfrecord-00143-of-01000_179': {(0,): 0.9777777777777777,
  (1,): 1.0,
  (2,): 0.9777777777777777,
  (3,): 0.9777777777777777,
  (4,): 1.0,
  (5,): 0.9666666666666667,
  (6,): 0.9777777777777777,
  (7,): 1.0,
  (8,): 0.9777777777777777,
  (9,): 1.0,
  (10,): 1.0,
  (11,): 1.0,
  (12,):